# Get gender

## Add `'gender'` key to database

In [233]:
import requests
from IPython.display import JSON
from gender_guesser_first_names import gender
from re import sub
from re import search
from re import UNICODE
import json
import country_converter as coco

In [267]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [235]:
import pandas as pd

In [6]:
DF = pd.DataFrame(df)
DFA = DF[~DF['author_id'].isna()]
DF.shape,DFA.shape

((57588, 13), (57576, 13))

In [7]:
DFB = DF[DF['author_id'].isna()].reset_index(drop=True)
DFB['author_id']=[f'unknown{str(x).zfill(2)}' for x in range(DFB.shape[0])]
DFB['author_id'] = DFB['author_id'].replace('unknown05','unknown04')

In [8]:
DF = pd.concat((DFA,DFB)).reset_index(drop=True)
del DFA
del DFB
DF.shape

(57588, 13)

In [9]:
def remove_initials(d):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    if (d.get('name') and d.get('name').get('value') ):
        first_names = d.get('name').get('value').split(', ')[-1].replace('-',' ').strip()
    elif d.get('full_name'):
        first_names = d.get('full_name').split(', ')[-1].replace('-',' ').strip()
    else:
        first_names = ''
    x = first_names
    x = sub(r'[\|†•]','',x).strip()
    x = sub(r'\.',' ',   x, UNICODE).strip()
    x = sub(r'\s\w\s','',x, UNICODE).strip()
    x = sub(r'^\w\s','',x, UNICODE).strip()
    x = sub(r'\s\w$','',x, UNICODE).strip()
    x = sub(r'^\w$','',x, UNICODE).strip()
    x = sub(r'^[A-Z]{2}$','',x, UNICODE)
    
    return x

In [10]:
DF['names'] = DF.apply(lambda d: remove_initials(d), axis='columns')

In [11]:
#DF = DF[DF['names'].apply(len)>0]
DF.shape

(57588, 14)

In [12]:
DF[DF['names'].apply(len)==0][:3]

,author_id,country,email_addresses,full_name,ids,institution,institution_id,name,papers,positions,profile_id,advisors,gender,names
12,J.Ospino.1,Spain,None,"Ospino, J.","[{'value': 'J.Ospino.1', 'schema': 'INSPIRE BA...","Salamanca U., IUFFyM",910114,"{'value': 'Ospino, J.', 'preferred_name': 'J. ...","[{'recid': 1781327, 'year': '2020', 'citation_...",[{'record': {'$ref': 'https://inspirehep.net/a...,1041300,[],unknown,
24,J.H.Pena.1,Venezuela,None,"Pena, J.M.","[{'value': 'J.H.Pena.1', 'schema': 'INSPIRE BA...","Caracas, U. Central",902716,"{'value': 'Pena, J.M.', 'preferred_name': 'J.M...","[{'recid': 781916, 'year': '2008', 'citation_c...",[{'record': {'$ref': 'https://inspirehep.net/a...,1052800,[],unknown,
29,A.J.Kalnay.1,Venezuela,None,"Kalnay, A.J.",None,Simon Bolivar U.,903205,None,"[{'recid': 93012, 'year': '1974', 'citation_co...",None,2204420,[],unknown,


In [13]:
DF[DF['names'].str.split().apply(lambda L: 1 in [len(x) for x in L])].names

Series([], Name: names, dtype: object)

In [14]:
DF[DF['names'].str.split().apply(lambda L: 2 in [len(x) for x in L])].names.unique()[:3]

array(['Le', 'Luis Ye', 'Kee Su'], dtype=object)

In [15]:
DFU = DF[['country']].drop_duplicates()

In [16]:
DFU['country_id'] = DFU['country'].astype(str).replace('Serbia and Montenegro','Serbia').apply(lambda c: coco.convert(names=c, to='ISO2')).replace('not found','')

None not found in regex


In [17]:
DF = DF.merge(DFU,on='country',how='left')

In [18]:
del DFU

In [19]:
DF['country_id'].unique()

array(['VE', 'CL', 'RW', 'ZA', 'US', 'ES', 'FR', 'CH', 'IT', 'DE', 'PE',
       'AR', 'UY', 'BR', 'EC', 'GB', 'CO', 'IL', 'BG', 'MX', 'NL', 'JP',
       'CA', 'SE', 'IR', 'IN', 'SI', 'BY', 'AU', 'RU', 'BE', 'HU', 'DK',
       '', 'SK', 'TH', 'CZ', 'JO', 'GR', 'CU', 'KW', 'FI', 'UA', 'AT',
       'TR', 'RS', 'PT', 'AM', 'KR', 'CN', 'MT', 'PL', 'EE', 'RO', 'HR',
       'DO', 'GT', 'IS', 'PY', 'PA', 'HN', 'PH', 'SV', 'PK', 'OM', 'CR',
       'NZ', 'GE', 'BO', 'KZ', 'VN', 'AE', 'NO', 'EG', 'TW', 'PR', 'JM',
       'LT', 'ID', 'LU', 'DZ', 'ET', 'SA', 'IE', 'CY', 'VA', 'MD', 'MK',
       'LB', 'BD', 'QA', 'MA', 'SG', 'MN', 'UZ', 'BS', 'BJ', 'LK', 'IQ',
       'AZ', 'SY', 'NG', 'PS', 'GH', 'KG', 'CG', 'ME', 'NE', 'BN', 'LV',
       'MY', 'MG', 'BT', 'TN', 'UG', 'BA'], dtype=object)

In [64]:
DFU = DF[['author_id','names','country','country_id']].rename({'names':'name'},axis='columns').drop_duplicates('author_id').reset_index(drop=True)
DFU.shape

(29979, 4)

In [65]:
D=gender.Detector(case_sensitive=False)
def force_get_gender(d,first_names='names'):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    gender = 'unknown'
    names = d.get(first_names)
    if names:
        if d.get('country'):
            gender = D.get_gender( names, country=d.get('country').strip() )
        else:
            gender = D.get_gender( names )
    return gender

In [66]:
DFU['gender'] = DFU.apply(lambda d: force_get_gender(d,first_names='name'), axis='columns')

In [67]:
DFU.shape

(29979, 5)

In [68]:
DFUA = DFU[DFU['name'].apply(len)>0].reset_index(drop=True)
DFUB = DFU[DFU['name'].apply(len)==0].reset_index(drop=True)
DFU.shape[0], DFUA.shape[0], DFUB.shape[0] + DFUA.shape[0],round( DFUB.shape[0] / DFU.shape[0],2)

(29979, 20125, 29979, 0.33)

In [71]:
DFUA.shape

(20125, 5)

In [72]:
DFUA.to_csv('/home/restrepo/Downloads/genderize.csv',index=False)

In [78]:
csv = 'data/genderize.csv'
DFUA = pd.read_csv(csv)

In [79]:
DFU = pd.concat((DFUA,DFUB)).reset_index(drop=True)

In [85]:
DFU['Gender'] = DFU['Gender'].fillna('unknown')
DFU['Gender Probability'] = DFU['Gender Probability'].fillna(0)
DFU['Gender Count'] = DFU['Gender Count'].fillna(0)
DFU

,author_id,name,country,country_id,gender,Gender,Gender Probability,Gender Count
0,H.Albrecht.1,Hartwig,Venezuela,VE,male,unknown,0.0,0.0
1,D.M.B.R.1,David Bellorin,Venezuela,VE,male,male,1.0,7693.0
2,D.F.Mundarain.1,Douglas,Chile,CL,male,male,1.0,707.0
3,R.Gaitan.2,Rolando,Venezuela,VE,male,male,1.0,845.0
4,J.Ntahompagaze.1,Joseph,Rwanda,RW,male,male,1.0,627.0
...,...,...,...,...,...,...,...,...
29974,P.P.Queiroz.2,,Brazil,BR,unknown,unknown,0.0,0.0
29975,unknown00,,Germany,DE,unknown,unknown,0.0,0.0
29976,unknown02,,Argentina,AR,unknown,unknown,0.0,0.0
29977,unknown08,,Mexico,MX,unknown,unknown,0.0,0.0


In [89]:
DFU[DFU['gender'].apply(lambda s: s in ['male','female'])].shape, DFU[DFU['Gender'].apply(lambda s: s in ['male','female'])].shape, 

((16968, 8), (19266, 8))

In [93]:
(DFU[ ( DFU['gender'].apply(lambda s: s in ['male','female']) ) & (DFU['Gender']=='unknown')].shape, 
 DFU[ ( DFU['Gender'].apply(lambda s: s in ['male','female']) ) & (DFU['gender']=='unknown')].shape)

((143, 8), (1817, 8))

In [100]:
DFU[ DFU['gender'] != DFU['Gender']  ].shape

(2870, 8)

In [110]:
DFU[ (DFU['gender'].apply(lambda s: s in ['male','female']) ) &
     (DFU['Gender'].apply(lambda s: s in ['male','female']) ) &
    (DFU['gender'] != DFU['Gender'])
][:50]

,author_id,name,country,country_id,gender,Gender,Gender Probability,Gender Count
70,L.L.Denmat.1,Le,France,FR,female,male,0.78,13210.0
99,C.C.Ciobirca.1,Costin Catalin,Venezuela,VE,male,female,1.00,6.0
142,J.C.Perez.Mosquera.1,Jean Carlos,Venezuela,VE,male,female,0.87,3998.0
297,R.M.Santilli.1,Ruggero Maria,United States,US,male,female,1.00,476674.0
349,J.P.Veiro.1,Jean Pierre,Venezuela,VE,male,female,0.87,3998.0
364,N.A.Lo.Presti.1,Nicola Adriano,France,FR,female,male,0.90,1822.0
402,W.L.Van.Neerven.1,Willy,Netherlands,NL,male,female,0.61,5932.0
436,R.Calaga.1,Buchi Rama Rao,United States,US,male,female,0.67,1306.0
448,J.Pestieau.1,Jean,Venezuela,VE,male,female,0.87,3998.0
543,Miha.Nemevsek.1,Miha,Italy,IT,male,female,0.88,534.0


In [203]:
author_id = DFU.author_id.loc[2]
author_id

'D.F.Mundarain.1'

In [247]:
pd.DataFrame(df)[pd.DataFrame(df)['full_name']=='Juárez W., S. Rebeca']

,author_id,country,email_addresses,full_name,ids,institution,institution_id,name,papers,positions,profile_id,advisors,gender
25929,S.R.Juarez.W.3,Mexico,None,"Juárez W., S. Rebeca",None,"Mexico, ESFM",907888,None,"[{'recid': 1516210, 'year': '2017', 'citation_...",None,None,[],female
33065,unknown10,Italy,None,"Juárez W., S. Rebeca",None,"SIP-OTBU/NE, Trieste",1241220,None,"[{'recid': 1800459, 'year': '2021', 'citation_...",None,None,[],female


In [261]:
DFU.columns

Index(['author_id', 'name', 'country', 'country_id', 'gender', 'Gender',
       'Gender Probability', 'Gender Count'],
      dtype='object')

In [268]:
d_gender = dict(zip( DFU['author_id'], DFU['Gender'] ) )
d_first_names = dict(zip( DFU['author_id'], DFU.rename({'name':'first_names'},axis='columns')['first_names'] ) )
d_prob   = dict(zip( DFU['author_id'], DFU['Gender Probability'] ) )
unknown = DF[DF['author_id'].str.contains('unknown')]
d_author_id = dict(zip(unknown['full_name'],unknown['author_id']))

In [269]:
kk = [d.update({'author_id': d_author_id[ d['full_name']  ]}) for d in df if d.get('author_id') is None ]

In [ ]:
for author_id in DFU['author_id']:
    kk = [d.update( {'first_names': d_first_names[author_id], 
                     'gender':      d_gender[author_id] ,
                     'gender_probability': d_prob[author_id]  } )
              for d in df if d.get('author_id') == author_id]

In [307]:
TMP  = pd.DataFrame(df)
TMPU = TMP.drop_duplicates('author_id').reset_index(drop=True)
TMPUI = TMPU[TMPU['first_names'].apply(len)>0].reset_index(drop=True)
TMPUA = TMPUI[TMPUI['gender_probability']>0.94].reset_index(drop=True)

print(f'TOTAL: {TMP.shape[0]}')
print(f'Unique autors: {TMPU.shape[0]}')
print(f'''Remove first names with only intials: {TMPUI.shape[0]} = \
        male:  {TMPU[TMPU['gender']=='male'].shape[0]} + female: {
                TMPU[TMPU['gender']=='female'].shape[0]} + unknown: {
                TMPU[TMPU['gender']=='unknown'].shape[0]}.
         ''')
print(f'Select found gender found probability from 95%: {TMPUA.shape[0]}')
print(f'female ratio at probability at 95%: { 
               round(TMPUA[TMPUA['gender']=='female'].shape[0]/(TMPUA[TMPUA['gender']=='female'].shape[0] + TMPUA[TMPUA['gender']=='male'].shape[0]),2)*100  }%')

TOTAL: 57588
Unique autors: 29979
Remove first names with only intials: 20125 =         male:  16013 + female: 3253 + unknown: 10713.
         
Select found gender found probability from 95%: 17634
female ratio at probability at 95%: 16.0%


In [308]:
f=open('data/inspire_LA.json','w')
json.dump(df,f)
f.close()

In [259]:
TMP.shape[0] , TMP[TMP['gender'].apply(lambda s: s in ['male','female'])].shape[0]

(57588, 39349)

In [118]:
DFU[DFU['Gender Probability']>0.].shape,DFU[DFU['Gender Probability']>=0.95].shape

((19266, 8), (17634, 8))

In [120]:
DFU[ DFU['gender'] != DFU['Gender']  ].shape

(2870, 8)

In [111]:
CO = pd.read_json('https://raw.githubusercontent.com/colav-playground/gender_guesser_first_names/refs/heads/main/data/test_gender.json')

In [ ]:
CO['country_id'] = CO['country'].astype(str).apply(lambda c: coco.convert(names=c, to='ISO2')).replace('not found','')

In [124]:
CO.to_csv('/home/restrepo/Downloads/test_gender.csv',index=False)

In [125]:
CO = pd.read_csv('data/test_gender.csv')

In [129]:
CO[ (CO['gender'].apply(lambda s: s in ['male','female']) ) &
     (CO['Gender'].apply(lambda s: s in ['male','female']) ) &
    (CO['gender'] != CO['Gender'])
].shape

(28, 7)

In [135]:
CO[ CO['Gender'] == 'unknown']

,names,gender,country,country_id,Gender,Gender Probability,Gender Count
96,Awar Simon,male,other_countries,NaN,unknown,0.0,0
104,Olufemi Adeyinka,male,other_countries,NaN,unknown,0.0,0
112,Francois,male,other_countries,NaN,unknown,0.0,0
115,Geoffrey,male,other_countries,NaN,unknown,0.0,0
117,Claire,female,other_countries,NaN,unknown,0.0,0
...,...,...,...,...,...,...,...
3818,Bethany Noel,female,other_countries,NaN,unknown,0.0,0
3819,Florian Jan Fietje,male,other_countries,NaN,unknown,0.0,0
3820,Yaniel,male,other_countries,NaN,unknown,0.0,0
3821,Lars Fredrik,male,other_countries,NaN,unknown,0.0,0


In [137]:
CO.shape

(3967, 7)

In [139]:
kk = CO[ CO['Gender'] == CO['gender'] ]

In [155]:
 kk[kk['Gender Probability']>0.94].shape[0] /kk.shape[0]

0.978488982161595

In [131]:
28/CO.shape[0]*100

0.7058230400806654

In [ ]:
f=open('data/inspire_LA.json','w')
json.dump(df,f)
f.close()

## Analysis

In [ ]:
import pandas as pd

In [ ]:
DF = pd.read_json('data/inspire_LA.json')

Unique authors ids

In [ ]:
DF.shape[0]

In [ ]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape[0]

In [ ]:
DF['first_names'] = DF['name'].str.get('value').astype(str).str.split(', ').str[-1]#.apply(
    #lambda s: sub('\w\.','',s)).str.replace('None','')

In [ ]:
DF['first_names'] = DF.apply(lambda row: str(row['full_name']).split(', ')[-1]  
                             if row['first_names']=='None' 
                             else row['first_names'] ,axis = 'columns')

In [ ]:
DF['gender'].unique()

In [ ]:
print(DF['gender'].shape[0])
total = DF['gender'].dropna().shape[0]
total

In [ ]:
male = DF[DF['gender'] == 'male'].shape[0]
female = DF[DF['gender'] == 'female'].shape[0]
unknown = DF[DF['gender'] == 'unknown'].shape[0]
unknown = unknown + DF[DF['gender'] == 'andy'].shape[0]

In [ ]:
assert male + female + unknown == total

In [ ]:
male+female

In [ ]:
male

In [ ]:
female

In [ ]:
round(female/male,2)

In [ ]:
unknown

In [ ]:
DFFM = DF[ (( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFFM.shape

In [ ]:
DFU = DF[ ~(( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFU.shape

In [ ]:
DFFM[['author_id','first_names','country','gender']].sample(50)

In [ ]:
#[['first_names','country','gender']]
DFU.sample(50)[['author_id','full_name','name','country','gender']]

check specific entries

In [ ]:
JSON([d for d in df if d.get('author_id')=='F.Fontanot.3'][0])

In [ ]:
import requests
au = requests.get('https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1')
if au.status_code == 200:
    au = au.json()

In [ ]:
len(au['hits']['hits'])

In [ ]:
au['links']['next']

In [ ]:
# https://github.com/inspirehep/rest-api-doc/issues/20#issuecomment-997932079
url='https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1&q=control_number:[1+TO+2852058]'
au = requests.get(url)
if au.status_code == 200:
    au = au.json()

In [ ]:
len(au['hits']['hits'])

In [ ]:
au.status_code

In [ ]:
au = requests.get(url)

In [ ]:
step = 10000
ini = 981872
end = ini + step
for i in range(187):
    print(f'control_number:[{ini} TO {end}]')
    ini = end + 1
    end = ini + step

In [ ]:
au.status_code

In [ ]:
len(au.json()['hits']['hits'])